In [203]:
%%capture
!pip install -r requirements-components-test.txt

In [204]:
import torch
import numpy as np

In [205]:
np.__version__

'1.25.2'

In [206]:
from torch import nn

In [207]:
torch.__version__

'2.3.0+cu121'

In [208]:
import pandas as pd

In [209]:
pd.__version__

'2.0.3'

In [210]:
import os
import requests
import logging
# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Configuration variables
config = {
    "REPO_URL": os.environ.get('REPO_URL', 'https://github.com/danilonicioka/mlops-workflow.git'),
    "CLONED_DIR": os.environ.get('CLONED_DIR', 'mlops-workflow'),
    "FILE_URL": os.environ.get('FILE_URL', 'https://raw.githubusercontent.com/razaulmustafa852/youtubegoes5g/main/Models/Stall-Windows%20-%20Stall-3s.csv'),
    "DVC_FILE_DIR": os.environ.get('DVC_FILE_DIR', 'data/external'),
    "DVC_FILE_NAME": os.environ.get('DVC_FILE_NAME', 'init_dataset.csv'),
    "BRANCH_NAME": os.environ.get('BRANCH_NAME', 'tests'),
    "BUCKET_NAME": os.environ.get('BUCKET_NAME', 'dvc-data'),
    "MINIO_URL": os.environ.get('MINIO_URL', 'localhost:9000'),
    "ACCESS_KEY": os.environ.get('ACCESS_KEY'),
    "SECRET_KEY": os.environ.get('SECRET_KEY'),
    "REMOTE_NAME": os.environ.get('REMOTE_NAME', 'minio_remote'),
    "GITHUB_USERNAME": os.environ.get('GITHUB_USERNAME'),
    "GITHUB_TOKEN": os.environ.get('GITHUB_TOKEN')
}

file_url = config["FILE_URL"]
local_file_path = config["DVC_FILE_NAME"]

try:
    # Request the file content
    response = requests.get(file_url)
    response.raise_for_status()

    # Save the file content locally
    with open(local_file_path, 'wb') as local_file:
        local_file.write(response.content)
    logger.info(f"Successfully downloaded file from {file_url} to {local_file_path}")
except requests.RequestException as e:
    # Log and raise any download errors
    logger.error(f"Failed to download file: {e}")
    raise

INFO:__main__:Successfully downloaded file from https://raw.githubusercontent.com/razaulmustafa852/youtubegoes5g/main/Models/Stall-Windows%20-%20Stall-3s.csv to init_dataset.csv


In [211]:
#colab_path = os.path.join('/content', local_file_path)
df = pd.read_csv(local_file_path)

In [212]:
df.columns

Index(['ID', 'Stall', 'Quality', 'Time', 'CQI1', 'CQI2', 'CQI3', 'cSTD CQI',
       'cMajority', 'c25 P', 'c50 P', 'c75 P', 'RSRP1', 'RSRP2', 'RSRP3',
       'pMajority', 'p25 P', 'p50 P', 'p75 P', 'RSRQ1', 'RSRQ2', 'RSRQ3',
       'qMajority', 'q25 P', 'q50 P', 'q75 P', 'SNR1', 'SNR2', 'SNR3',
       'sMajority', 's25 P', 's50 P', 's75 P'],
      dtype='object')

In [213]:
df = df.replace([' ', '-',np.nan], 0) # There are null values
#df = df.replace([' ', '-',np.nan], np.nan)

In [214]:
# Selective columns for mean calculation
columns_to_convert = ['CQI1', 'CQI2', 'CQI3', 'cSTD CQI',
       'cMajority', 'c25 P', 'c50 P', 'c75 P', 'RSRP1', 'RSRP2', 'RSRP3',
       'pMajority', 'p25 P', 'p50 P', 'p75 P', 'RSRQ1', 'RSRQ2', 'RSRQ3',
       'qMajority', 'q25 P', 'q50 P', 'q75 P', 'SNR1', 'SNR2', 'SNR3',
       'sMajority', 's25 P', 's50 P', 's75 P']
df[columns_to_convert] = df[columns_to_convert].astype(float)

# Replace np.nan with mean values for selective columns
df[columns_to_convert] = df[columns_to_convert].fillna(df[columns_to_convert].mean())

# Display the modified DataFrame
#print(df)

In [215]:
# Check which columns contain np.nan values
columns_with_nan = df.isna().any()
# Display the columns with np.nan values
print(columns_with_nan)

ID           False
Stall        False
Quality      False
Time         False
CQI1         False
CQI2         False
CQI3         False
cSTD CQI     False
cMajority    False
c25 P        False
c50 P        False
c75 P        False
RSRP1        False
RSRP2        False
RSRP3        False
pMajority    False
p25 P        False
p50 P        False
p75 P        False
RSRQ1        False
RSRQ2        False
RSRQ3        False
qMajority    False
q25 P        False
q50 P        False
q75 P        False
SNR1         False
SNR2         False
SNR3         False
sMajority    False
s25 P        False
s50 P        False
s75 P        False
dtype: bool


In [216]:
df['Stall'].replace('Yes', 1, inplace=True)
df['Stall'].replace('No', 0, inplace=True)

In [217]:
print(df)

          ID  Stall   Quality      Time  CQI1  CQI2  CQI3  cSTD CQI  \
0      4P7s2      0    hd2160  16:14:29  13.0  13.0  13.0  0.000000   
1      4P7s2      0    hd2160  16:14:30  13.0  13.0  13.0  0.000000   
2      4P7s2      0    hd2160  16:14:31  13.0  13.0  13.0  0.000000   
3      4P7s2      0    hd2160  16:14:32  13.0  13.0  12.0  0.471405   
4      4P7s2      0    hd2160  16:14:33  12.0  14.0  12.0  0.942809   
...      ...    ...       ...       ...   ...   ...   ...       ...   
2688  5Po26s      1  unknown,  17:43:18  14.0  14.0  14.0  0.000000   
2689  4Po26s      1  unknown,  17:43:23   0.0   0.0   0.0  0.000000   
2690  4Po26s      1   hd1440,  17:43:33   0.0   0.0   0.0  0.000000   
2691   4I27s      1  unknown,  10:52:04   9.0   9.0   9.0  0.000000   
2692   4I27s      1   hd1440,  10:52:16   9.0   9.0   9.0  0.000000   

      cMajority  c25 P  ...  q25 P  q50 P  q75 P  SNR1  SNR2  SNR3  sMajority  \
0          13.0   13.0  ...   -9.5   -7.0   -7.0  12.0  12.0   7.0

In [218]:
df.columns

Index(['ID', 'Stall', 'Quality', 'Time', 'CQI1', 'CQI2', 'CQI3', 'cSTD CQI',
       'cMajority', 'c25 P', 'c50 P', 'c75 P', 'RSRP1', 'RSRP2', 'RSRP3',
       'pMajority', 'p25 P', 'p50 P', 'p75 P', 'RSRQ1', 'RSRQ2', 'RSRQ3',
       'qMajority', 'q25 P', 'q50 P', 'q75 P', 'SNR1', 'SNR2', 'SNR3',
       'sMajority', 's25 P', 's50 P', 's75 P'],
      dtype='object')

In [219]:
X = df[['CQI1', 'CQI2', 'CQI3', 'cSTD CQI',
       'cMajority', 'c25 P', 'c50 P', 'c75 P', 'RSRP1', 'RSRP2', 'RSRP3',
       'pMajority', 'p25 P', 'p50 P', 'p75 P', 'RSRQ1', 'RSRQ2', 'RSRQ3',
       'qMajority', 'q25 P', 'q50 P', 'q75 P', 'SNR1', 'SNR2', 'SNR3',
       'sMajority', 's25 P', 's50 P', 's75 P']].values

In [220]:
y = df['Stall'].values

In [221]:
X.shape, y.shape

((2693, 29), (2693,))

In [222]:
import numpy as np

In [223]:
import sklearn
from sklearn.model_selection import train_test_split

In [224]:
sklearn.__version__

'1.2.2'

In [225]:
import imblearn
from imblearn.over_sampling import SMOTE

In [226]:
imblearn.__version__

'0.11.0'

In [227]:
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [228]:
X[0]

array([ 13. ,  13. ,  13. ,   0. ,  13. ,  13. ,  13. ,  13. , -76. ,
       -76. , -81. , -76. , -78.5, -76. , -76. ,  -7. ,  -7. , -12. ,
        -7. ,  -9.5,  -7. ,  -7. ,  12. ,  12. ,   7. ,  12. ,   9.5,
        12. ,  12. ])

In [229]:
from sklearn.preprocessing import StandardScaler
import joblib

scaler = StandardScaler()
fit_scaler = scaler.fit(X)
joblib.dump(fit_scaler, 'scaler.save')
X = fit_scaler.transform(X)
X.shape

(3524, 29)

In [230]:
X = torch.from_numpy(X).type(torch.float32)
y = torch.from_numpy(y).type(torch.float32)

In [231]:
X.shape

torch.Size([3524, 29])

In [232]:
X[0]

tensor([ 1.2961,  1.2947,  1.3103, -0.8068,  1.2950,  1.5510,  1.3164,  1.2005,
         1.3405,  1.3391,  1.0186,  1.3402,  1.2126,  1.3393,  1.3140,  0.5796,
         0.5772, -0.5194,  0.5746,  0.0798,  0.5754,  0.5401,  0.4581,  0.4473,
        -0.1917,  0.4490,  0.1823,  0.4479,  0.4039])

In [233]:
y.shape

torch.Size([3524])

In [234]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42
)

print("X_train:", X_train[:1],"\nX_train_shape:", X_train[:1].shape,"\nX_test:", X_test[:1],"\nX_test_shape:",X_test[:1].shape, "\ny_train:", y_train[:1],"\ny_test:", y_test[:1])

X_train: tensor([[ 0.3143,  1.2947, -0.3344,  1.4486,  0.3206,  0.1346,  0.3289,  0.6980,
          0.7602,  0.7593,  0.7615,  0.7602,  0.7921,  0.7594,  0.7331,  0.3591,
          0.3575, -0.0798,  0.3551,  0.1889,  0.3558,  0.3141,  1.1001,  1.0871,
          0.9555,  1.0899,  1.0824,  1.0884,  1.0531]]) 
X_train_shape: torch.Size([1, 29]) 
X_test: tensor([[ 1.9507,  1.6197,  1.6393, -0.2894,  1.6199,  1.9051,  1.6455,  1.7030,
          1.2760,  1.4035,  1.2756,  1.2757,  1.3096,  1.2749,  1.3140,  1.4618,
          1.4557,  1.4588,  1.4527,  1.4977,  1.4540,  1.4438,  1.7421,  1.7269,
          1.7203,  1.7309,  1.7897,  1.7289,  1.7023]]) 
X_test_shape: torch.Size([1, 29]) 
y_train: tensor([0.]) 
y_test: tensor([0.])


In [235]:
print("X_train:", X_train[:1])

X_train: tensor([[ 0.3143,  1.2947, -0.3344,  1.4486,  0.3206,  0.1346,  0.3289,  0.6980,
          0.7602,  0.7593,  0.7615,  0.7602,  0.7921,  0.7594,  0.7331,  0.3591,
          0.3575, -0.0798,  0.3551,  0.1889,  0.3558,  0.3141,  1.1001,  1.0871,
          0.9555,  1.0899,  1.0824,  1.0884,  1.0531]])


In [236]:
torch.save(X_train, "/tmp/X_train.pt")

In [237]:
X_train_loaded = torch.load("/tmp/X_train.pt")
print("X_train:", X_train_loaded[:1])

X_train: tensor([[ 0.3143,  1.2947, -0.3344,  1.4486,  0.3206,  0.1346,  0.3289,  0.6980,
          0.7602,  0.7593,  0.7615,  0.7602,  0.7921,  0.7594,  0.7331,  0.3591,
          0.3575, -0.0798,  0.3551,  0.1889,  0.3558,  0.3141,  1.1001,  1.0871,
          0.9555,  1.0899,  1.0824,  1.0884,  1.0531]])


In [238]:
type(X_train)

torch.Tensor

In [239]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [240]:
# Build model with non-linear activation function
from torch import nn
class InteruptionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=29, out_features=200)
        self.layer_2 = nn.Linear(in_features=200, out_features=100)
        self.layer_3 = nn.Linear(in_features=100, out_features=1)
        self.relu = nn.ReLU() # <- add in ReLU activation function
        # Can also put sigmoid in the model
        # This would mean you don't need to use it on the predictions
        # self.sigmoid = nn.Sigmoid()

    def forward(self, x):
      # Intersperse the ReLU activation function between layers
       return self.layer_3(self.relu(self.layer_2(self.relu(self.layer_1(x)))))

model_3 = InteruptionModel().to(device)
print(model_3)

InteruptionModel(
  (layer_1): Linear(in_features=29, out_features=200, bias=True)
  (layer_2): Linear(in_features=200, out_features=100, bias=True)
  (layer_3): Linear(in_features=100, out_features=1, bias=True)
  (relu): ReLU()
)


In [241]:
#model_3.state_dict()

In [242]:
# Setup loss and optimizer
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_3.parameters(), lr=0.0001)

In [243]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [244]:
# Fit the model
torch.manual_seed(42)
epochs = 3500

# Put all data on target device
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)


for epoch in range(epochs):
    # 1. Forward pass
    y_logits = model_3(X_train).squeeze()

    y_pred = torch.round(torch.sigmoid(y_logits)) # logits -> prediction probabilities -> prediction labels

    # 2. Calculate loss and accuracy
    loss = loss_fn(y_logits, y_train) # BCEWithLogitsLoss calculates loss using logits
    acc = accuracy_fn(y_true=y_train,
                      y_pred=y_pred)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model_3.eval()
    with torch.no_grad():
      # 1. Forward pass
        test_logits = model_3(X_test).squeeze()
        #print(test_logits.shape)
        test_pred = torch.round(torch.sigmoid(test_logits)) # logits -> prediction probabilities -> prediction labels
        # 2. Calcuate loss and accuracy
        test_loss = loss_fn(test_logits, y_test)
        test_acc = accuracy_fn(y_true=y_test,
                             y_pred=test_pred)


    # Print out what's happening
    if epoch % 500 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Accuracy: {test_acc:.2f}%")

Epoch: 0 | Loss: 0.69251, Accuracy: 50.66% | Test Loss: 0.68589, Test Accuracy: 55.46%
Epoch: 500 | Loss: 0.47526, Accuracy: 77.76% | Test Loss: 0.49838, Test Accuracy: 75.18%
Epoch: 1000 | Loss: 0.37302, Accuracy: 83.43% | Test Loss: 0.44735, Test Accuracy: 80.43%
Epoch: 1500 | Loss: 0.26883, Accuracy: 89.61% | Test Loss: 0.41710, Test Accuracy: 82.70%
Epoch: 2000 | Loss: 0.17973, Accuracy: 94.40% | Test Loss: 0.42712, Test Accuracy: 84.54%
Epoch: 2500 | Loss: 0.12543, Accuracy: 96.81% | Test Loss: 0.46299, Test Accuracy: 85.25%
Epoch: 3000 | Loss: 0.08981, Accuracy: 98.01% | Test Loss: 0.51030, Test Accuracy: 84.54%


In [245]:
model_3.eval()
with torch.no_grad():
     y_preds = torch.round(torch.sigmoid(model_3(X_test))).squeeze()

In [246]:
y_preds.shape,y_test.shape

(torch.Size([705]), torch.Size([705]))

In [247]:
if device == "cuda":
  predictions = y_preds.cpu().numpy() #if it is cuda, then this, otherwise y_pred.numpy()
  true_labels = y_test.cpu().numpy()
else:
  predictions = y_preds.numpy()
  true_labels = y_test.numpy()

In [248]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score,fbeta_score

print("=== Confusion Matrix ===")
print(confusion_matrix(true_labels, predictions))
print('\n')


print("=== Score ===")
accuracy = accuracy_score(true_labels, predictions)
print('Accuracy: %f' % accuracy)

precision = precision_score(true_labels,  predictions, average='weighted')
print('Precision: %f' % precision)
recall = recall_score(true_labels, predictions, average='weighted')
print('Recall: %f' % recall)

microf1 = f1_score(true_labels, predictions, average='micro')
print('Micro F1 score: %f' % microf1)
macrof1 = f1_score(true_labels, predictions, average='macro')
print('Macro F1 score: %f' % macrof1)

=== Confusion Matrix ===
[[310  60]
 [ 46 289]]


=== Score ===
Accuracy: 0.849645
Precision: 0.850493
Recall: 0.849645
Micro F1 score: 0.849645
Macro F1 score: 0.849512


In [249]:
target_names = ['No-Stall', 'Stall']
# Print precision-recall report
print(classification_report(true_labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

    No-Stall       0.87      0.84      0.85       370
       Stall       0.83      0.86      0.85       335

    accuracy                           0.85       705
   macro avg       0.85      0.85      0.85       705
weighted avg       0.85      0.85      0.85       705



In [250]:
#Done